In [ ]:
pip install transformers datasets torch torchvision
# installing some req libraries

In [ ]:
# Installing req packages
!pip install transformers[torch] accelerate -U
import os  #used for interacting with the operating system, such as reading file paths.
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset as TorchDataset
from PIL import Image
import torch
'''ViTFeatureExtractor: Used for preprocessing images for the Vision Transformer (ViT) model.
AutoTokenizer: Automatically selects the appropriate tokenizer for the text decoder model.
Seq2SeqTrainer: A Trainer class specifically designed for sequence-to-sequence models.
Seq2SeqTrainingArguments: Arguments specific to training sequence-to-sequence models.'''

In [ ]:
# Creating Datasett
class ImageTextDataset(TorchDataset):
    def __init__(self, img_dir, text_dataset): #cons
        self.img_dir = img_dir   # dir containg image file
        self.text_dataset = text_dataset # text fike
        self.feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
        # processing the image
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        #tokenizing data

    def __len__(self):
        return len(self.text_dataset)
        # returns the number of samples in this dataset

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.text_dataset[idx]['Image'])  #cons image path
        try:   # trying to open the image and convert it into RGB
            image = Image.open(img_path).convert("RGB")
            encoding = self.feature_extractor(images=image, return_tensors="pt")
            pixel_values = encoding['pixel_values'].squeeze()
            # Extracts the pixel values from the encoding and removes the batch dimension.


        except FileNotFoundError:
            print(f"Warning: Image not found at {img_path}. Skipping sample.")
            return None  # Or return a placeholder




        # img_path = os.path.join(self.img_dir, self.text_dataset[idx]['Image'])
        # image = Image.open(img_path).convert("RGB")
        # encoding = self.feature_extractor(images=image, return_tensors="pt")
        # pixel_values = encoding['pixel_values'].squeeze()  # Remove batch dimension

        text = self.text_dataset[idx]['Text']
        labels = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=64).input_ids
        labels = labels.squeeze()  # Remove batch dimension
        # Tokenizes the text, padding or truncating it to a maximum length of 64 tokens, and removes the batch dimension.

        return {
            "pixel_values": pixel_values,
            "labels": labels
        }


In [ ]:
Lo
from datasets import load_dataset, DatasetDict

# Loading the dataset
dataset = load_dataset("AY027/MMADE")

# Split the dataset into train and test
split_dataset = dataset['train'].train_test_split(test_size=0.2)
train_texts = split_dataset['train']
test_texts = split_dataset['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_dir = "/content/drive/MyDrive/MMADE/multimodal adr"
train_dataset = ImageTextDataset(img_dir=img_dir, text_dataset=train_texts)
test_dataset = ImageTextDataset(img_dir=img_dir, text_dataset=test_texts)


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
print(test_dataset[0])

{'pixel_values': tensor([[[ 0.1059, -0.0353, -0.1765,  ...,  0.2078,  0.2078,  0.2784],
         [ 0.0824, -0.0353, -0.1608,  ...,  0.2078,  0.2157,  0.2863],
         [ 0.0431, -0.0588, -0.1608,  ...,  0.1922,  0.2000,  0.2392],
         ...,
         [-0.3098, -0.3020, -0.3098,  ...,  0.3804,  0.3569,  0.4196],
         [-0.2941, -0.2863, -0.2941,  ...,  0.3725,  0.3490,  0.4118],
         [-0.2314, -0.2314, -0.2314,  ...,  0.4118,  0.3804,  0.4353]],

        [[ 0.0902, -0.0588, -0.2078,  ...,  0.0588,  0.0745,  0.1529],
         [ 0.0667, -0.0667, -0.1922,  ...,  0.0510,  0.0824,  0.1608],
         [ 0.0196, -0.0902, -0.1922,  ...,  0.0510,  0.0745,  0.1294],
         ...,
         [-0.2314, -0.2235, -0.2392,  ...,  0.2863,  0.2784,  0.3333],
         [-0.2392, -0.2314, -0.2392,  ...,  0.2941,  0.2706,  0.3333],
         [-0.1843, -0.1843, -0.1843,  ...,  0.3255,  0.3098,  0.3725]],

        [[ 0.4118,  0.3020,  0.2000,  ..., -0.1137, -0.0902,  0.0196],
         [ 0.4039,  0.3176, 

In [ ]:
# Loading the modell
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("google/vit-base-patch16-224-in21k", "bert-base-uncased")
# VisionEncoderDecoderModel: This class combines a vision encoder (like ViT) with a text decoder (like BERT).
# This method initializes the model by loading a pretrained Vision Transformer (ViT) as the encoder and a pretrained BERT model as the decoder.


# Print the bos_token_id and eos_token_id to check if they're None
print(f"bos_token_id: {model.decoder.config.bos_token_id}")
# bos_token_id: The "beginning of sentence" token ID. This is used to signal the start of a sequence.

print(f"eos_token_id: {model.config.eos_token_id}")
# eos_token_id: The "end of sentence" token ID. This signals the end of a sequence.

# Explicitly set the decoder start token ID to a suitable value if both bos and eos are None
if model.decoder.config.bos_token_id is None and model.config.eos_token_id is None:
    # Use the pad token as a safe alternative for decoder start token
    model.config.decoder_start_token_id = model.decoder.config.pad_token_id
    #  If both are None, set the decoder_start_token_id to the pad_token_id of the decoder. The pad_token_id is a safe choice
    # because it indicates a padding token, which is often available in pretrained models.
    print("Both bos_token_id and eos_token_id were None. Setting decoder_start_token_id to pad_token_id.")

model.config.pad_token_id = model.decoder.config.pad_token_id
# pad_token_id: Ensures the model's pad_token_id is set to the decoder's pad_token_id.
#   This is necessary for correct padding during training and evaluation.

model.config.vocab_size = model.config.decoder.vocab_size
# vocab_size: Sets the model's vocab_size to match the decoder's vocabulary size.
#     This is important because the decoder's output space must match its vocabulary.

# Ensure the decoder_start_token_id is correctly set
assert model.config.decoder_start_token_id is not None, "decoder_start_token_id is not set"

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

bos_token_id: None
eos_token_id: None
Both bos_token_id and eos_token_id were None. Setting decoder_start_token_id to pad_token_id.


In [ ]:
training_args = Seq2SeqTrainingArguments(  # configuration class from the Hugging Face transformers library that allows
                                         #  to specify various settings for training a sequence-to-sequence model using the Seq2SeqTrainer
    predict_with_generate=True,  # enables the generation of sequences during evaluation
                                           #  produce predictions (e.g., text sequences) from the input data
    evaluation_strategy="steps", #  model will be evaluated at regular intervals
    per_device_train_batch_size=4, # The batch size to use for training on each device
    per_device_eval_batch_size=4,  # The batch size to use for evaluation on each device.
    logging_steps=10,  # number of training steps between logging events
    save_steps=10,  # The number of training steps between checkpoints.
    eval_steps=10,   # The number of training steps between evaluations.
    output_dir="./results",
    overwrite_output_dir=True,
    save_total_limit=3,
                     # Limits the total number of checkpoints to save. If the limit is exceeded, the oldest checkpoint will be deleted.
                     #    This helps manage disk space by only keeping a specified number of recent checkpoints.
    num_train_epochs=3
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,   # with VisionEncoderDecoderModel
    args=training_args,   # training arguments
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
# there are some imageses missing in the folder ...
import torch
from transformers import default_data_collator # Import the default data collator

def custom_data_collator(features):  #  a custom function to handle potential issues with the input features.
    # Handle potential None values in the list
    valid_features = [f for f in features if f is not None] # filters out any None values from the features

    # Handle potential None values within valid features
    for feature in valid_features: # loop iterates over each valid feature (data sample).
        for key in feature.keys(): #  nested loop iterates over each key in the current feature's dictionary.
            if feature[key] is None:
                feature[key] = torch.tensor([])  # Or any appropriate default value

    return default_data_collator(valid_features) # Use the imported default data collator on valid features

trainer.data_collator = custom_data_collator
# This attribute of the Seq2SeqTrainer instance specifies the function to be used for collating data into batches during training and evaluation.


In [ ]:
trainer.train()  #  initiates the training loop for the model


Step,Training Loss,Validation Loss
10,4.316100,3.743747
20,3.619900,3.207748
30,3.139200,2.935901
40,2.688400,2.800291
50,2.644400,2.679680
60,2.502600,2.612618
70,2.575400,2.553284
80,2.592600,2.449193
90,2.499200,2.380542
100,2.320000,2.303218


KeyboardInterrupt: 

In [ ]:
from transformers import TrainerCallback, TrainingArguments, Trainer

class DebugCallback(TrainerCallback):
    def on_step_begin(self, args: TrainingArguments, state, control, **kwargs):
        if state.global_step == 0:
            # Access the dataloader from the trainer instance
            dataloader = trainer.get_train_dataloader()  # Access dataloader from the trainer
            first_batch = next(iter(dataloader))
            print("First batch:", first_batch)

# Add the DebugCallback to your trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[DebugCallback()]
)

trainer.train()

In [ ]:
from transformers import TrainerCallback, TrainingArguments, Trainer

class DebugCallback(TrainerCallback):
    def on_step_begin(self, args: TrainingArguments, state, control, **kwargs):
        if state.global_step == 0:
            # Access the dataloader from the trainer instance
            dataloader = trainer.get_train_dataloader()  # Access dataloader from the trainer
            first_batch = next(iter(dataloader))
            print("First batch:", first_batch)
            # Check for None values in the batch
            for k, v in first_batch.items():
                if v is None:
                    print(f"Warning: Key '{k}' has a None value.")

# Add the DebugCallback to your trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[DebugCallback()]
)

trainer.train()

In [ ]:
from transformers import TrainerCallback, TrainingArguments, Trainer

class DebugCallback(TrainerCallback):
    def on_step_begin(self, args: TrainingArguments, state, control, **kwargs):
        if state.global_step == 0:
            # Access the dataloader from the trainer instance
            dataloader = trainer.get_train_dataloader()  # Access dataloader from the trainer
            first_batch = next(iter(dataloader))
            print("First batch:", first_batch)
            # Check for None values in the batch and their types
            for k, v in first_batch.items():
                if v is None:
                    print(f"Warning: Key '{k}' has a None value.")
                else:
                    print(f"Key '{k}' has type: {type(v)}")  # Print the type of each value

# Add the DebugCallback to your trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[DebugCallback()]
)

trainer.train()